# Change Detection - Image Ratio Demo
This notebook detects change between satellite images taken at subsequent times over the same location.

The methods are calibrated on [Sentinel-2](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/overview) pre-post disaster imagery from the Caribbean and [Copernicus Emergency Mapping Service (EMS)](https://emergency.copernicus.eu/mapping/map-of-activations-rapid#zoom=3&lat=29.18235&lon=-70.57787&layers=BT00) ground truth damage assessment data. Section 4 allows you to evaluate predicted damages against reported damages for locations covered by EMS. The models were built using the [Descartes Labs](https://www.descarteslabs.com/) platform. If you wish to re-train the models or have more flexibility in your parameters, plots or methods, please visit the notebooks for each method at [thresholding.ipynb](./thresholding.ipynb) and [unet_classifier.ipynb](./unet_classifier.ipynb).
### Contents
- 1 - [Visualise Subsequent Imagery](#visualiseImagery)
- 2 - [Detect Change - Thresholding Method](#thresholding)
- 3 - [Detect Change - Trained Classifier Method](#classifier)
- 4 - [Evaluate Methods Against Ground Data](#evaluate)
____________
## Initialisation - Choose Location
Firstly we need to choose a location for which to detect change. You may either choose one of the locations with pre-assigned variables from the list displayed when running the first cell, or you can choose a new location by selecting 'New Location' which will then prompt you to provide variables of your own when running the second cell. Pre-set locations are the following:
- Training Set: Roseau, Dominica (pre-post hurricane Maria); Abricots, Haiti (pre-post hurricane Matthew)
- Test Set: Jeremie, Haiti (pre-post hurricane Matthew); Port Salut, Haiti (pre-post hurricane Matthew)
- Informal Settlements: Bidonville Killick Stenio Vincent, Port-au-Prince, Haiti (2018 to 2019); Parry Town, Ocho Rios, Jamaica (2018 to 2019)
- High Resolution: Hidalgo County, Texas, USA (2016 to 2019)

In [2]:
import ir_demo_functions as dfn # Import functions from demoFunction.py
location = dfn.chooseLocation() # Generate pre-defined location list
location # Display location dropdown

Dropdown(description='Location:', options=(('train - Roseau, Dominica', 0), ('train - Abricots, Haiti', 1), ('…

In [3]:
# Assign variables for chosen location - 'updates' caters for 'New Location' variable changing
variables, updates = dfn.assignVariables(location) 

Button(description='Show variables', style=ButtonStyle())

Output()

In [4]:
# Submits input variables for new location (if not new location - no action)
variables, updates = dfn.submitNewLocation(variables, updates)

_____________
<a id='visualiseImagery'></a>
## 1 - Visualise Imagery
We'll start by displaying before and after images for the chosen location. Two pointers here:
- You'll need to click the magic markers below the map to scale the imagery band colours properly. You can also untick the 'After' box to toggle between pre and post disaster.
- If you have defined a new location and no imagery appears for your location, try increasing the cloud fraction, but check this does not lead to overly cloudy images which may affect detection performance. Alternatively try changing or widening the requested dates, it is not uncommon to span several months for a good image.

In [5]:
# Create map with before and after images for specified location
dfn.beforeAfterImages(variables)


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


________
<a id='thresholding'></a>
## 2 - Detect Change - Ratio Thresholding
Next, let's take the logarithmic ratio of images for selected bands and display detected change superimposed on the image for time 2. Beneath the plot is a slider allowing you to vary the thresholds within which we are detecting change. If damage assessment data is available for the location this will be overlayed for a qualitative comparison.

> The optimal threshold interval for detecting building change has been determined as 0.01-0.1. However, due to differences in lighting between repeat images this may need adjsuted according to location. As the ratio is logarithmic, if the second image is considerably darker than the first, the appropriate thresholding may even be negative.

Detection interval equation for RGB (red, green and blue bands) where, for example, r1 denotes pixel value for red band at time 1:  $ threshold < \log\left(\frac{r2}{r1} \times \frac{b2}{b1} \times \frac{g2}{g1}\right) < cap  $

In [6]:
# Function to detect change through thresholding for location
plotVars, variables = dfn.thresholding(variables)

`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jup…

FloatRangeSlider(value=(0.001, 0.1), description='Filter ratios', max=0.5, min=-0.5, step=0.01)

Run box below to display updated detection result


In [7]:
# Re-plot detected change according to slider values
plotVars = dfn.plotChange(plotVars)

You may observe 2 main limitations with this method.
- Lack of small scale change detections due to 10x10m resolution of Sentinel Imagery. Try re-running the notebook selecting the high resolution location to see what is possible where 1x1m resolution is available.
- Sensitivity of thresholding to image lighting and changes in areas other than buildings, due to effect on ratio values. If threshold is unadjusted or mask is not applied to ocean this leads to false detections. To mitigate this limitation we trained a [classifier](#classifier) in the next section to recognise the evidence of damage buildings within the image ratio rather than relying on simple thresholding.

____________________
<a id='classifier'></a>
## 3 - Detect Change - Ratio Classifier
To avoid the drawbacks of thresholding we trained a Convolutional Neural Network (CNN) with a [U-Net](https://arxiv.org/abs/1505.04597) architecture to identify building change from the pixel ratios between before/after Sentinel-2 imagery. This model evaluates change per 'image tile' of which there will be many within your area of interest. Therefore, first let's draw an adjustable polygon over the desired area. Then, each corresponding tile will individually be fed in to the model for assessing change detection. Bare in mind the larger the area the longer it will take to run the model for that area.
> One could question the decision not to just increase tilesize. However not only does this method make the evaluation area more flexible, but also the model does not cater well for tile sizes larger than that for which it was trained due to the input layer structure.

In [8]:
m3, testPoly = dfn.drawPolygon(variables) # Get map upon which to draw polygon for assessment
m3 # Display map - When adjusting polygon please maintain geometry (e.g. bottom left as bottom left)


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [9]:
# Detect change for all tiles at least partly within polygon
classified, variables = dfn.classifyDamage(testPoly, variables, m3)

Longitude Columns:   0%|          | 0/2 [00:00<?, ?it/s]

Number of tiles requested: 4 . Approximately 32 seconds on 16GB RAM.

Job ID: dc6e4adaac3656745bccbbe033967d0e54811878ca2466f9
[######] | Steps: 93/93 | Stage: SUCCEEDED                                    


Longitude Columns:  50%|█████     | 1/2 [00:08<00:08,  8.20s/it]


Job ID: 575a039608708a1f971baf8702c2632be241d0e1e777fbbd
[######] | Steps: 93/93 | Stage: SUCCEEDED                                    


Longitude Columns:   0%|          | 0/2 [00:00<?, ?it/s]


Job ID: 70e608531777822d7addc61298ceb1d5dbc96e130b40c017
[######] | Steps: 93/93 | Stage: SUCCEEDED                                    


Longitude Columns:  50%|█████     | 1/2 [00:07<00:07,  7.84s/it]


Job ID: 9605d45615c69ed3ddd3de67e802b3c895b2282b92d09932
[######] | Steps: 93/93 | Stage: SUCCEEDED                                    


Latitude Rows: 100%|██████████| 2/2 [00:31<00:00, 15.86s/it]


_________________
<a id='evaluate'></a>
## 4 - Evaluate Methods Against Ground Data
After qualitatively assessing the ratio method's performance in the previous section, we will now quantify the accuracy of our methods for detecting building damage against Copernicus EMS post-disaster damage assessments where available.

Accuracy is determined by evaluating the correspondance between detected changed pixels and damaged building footprints. The metrics are as follows:
- Precision (proportion of damage detected): $P = \frac{True Positives}{True Positives + False Positives}$
- Recall (proportion of detections corresponding to damage): $R = \frac{True Positives}{True Positives + False Negatives}$
- F1 Score: $F1 = 2x\frac{P*R}{P+R}$

We will first plot a map evaluating thresholding accuracy, and then for classifier accuracy. Assessing classifier accuracy or accuracy for a smaller area will be much faster.

### Thresholding Accuracy
Evaluation time increases with number of pixels having detected change.

In [10]:
# Function evaluating thresholding accuracy from detected changes in section 2
dfn.thresholdingAccuracy(plotVars,variables) 

Extracting detections from image for evaluation.

Job ID: 1a18fc6c4c3adaed7c6bec2f4358b4ed3a570ece1f46e635
[######] | Steps: 107/107 | Stage: SUCCEEDED                                  

2038it [00:03, 566.76it/s]
Pixels evaluated:   0%|          | 6/2006 [00:00<00:40, 49.66it/s]

Changed pixels: 2006 
Damaged buildings: 392


Pixels evaluated: 100%|██████████| 2006/2006 [00:43<00:00, 46.26it/s]


Precision: 0.5535714285714286 
Recall: 0.36839481555333997 
F1 score: 0.4423867915419523



`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


### Classifier Accuracy

In [11]:
# Function evaluating classifier accuracy from detected changes in section 3
dfn.classifierAccuracy(classified,variables)

  9%|▉         | 27/300 [00:00<00:02, 132.46it/s]

Changed pixels: 300 
Damaged buildings: 135


100%|██████████| 300/300 [00:02<00:00, 133.42it/s]


Accuracy: 0.7333333333333333 
Recall: 0.9333333333333333 
F1 score: 0.8213333333333334



`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


Thanks for checking out the image ratio demo! Remember to have a look at the individual [thresholding](./thresholding.ipynb) and [unet_classifer](./unet_classifier.ipynb) notebooks to re-train models or take the analysis further.

# --------------- END ------------------------